In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_values_standardized.csv"))

df

In [ ]:
def plot(_d, patient_group='all', lab=None):
    d = _d.copy()
    
    """
    DATA PREPROCESSING...
    """
    d = d.drop(columns=['Unnamed: 0'])
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d[d.scale == 'original']
    d = d[d.setting != 'never']
    d = d[d.setting == patient_group]
    d = d.rename(columns={
        'wave': 'Wave'
    })
    
    d17 = d[(1 <= d.days_since_positive) & (d.days_since_positive <= 7)]
    d = d[(d.days_since_positive == 0) | (d.days_since_positive == 1) | (d.days_since_positive == 7)]

    d.days_since_positive = d.days_since_positive.apply(lambda x: f"Day {x}")
    d17.days_since_positive = d17.days_since_positive.apply(lambda x: f"Day {x}")
    
    """
    DAY 1-7 AVERAGE
    """
    d17['mean'] = d17['mean'] * d17['total_n']
    d17 = d17.groupby(['Lab', 'setting', 'Wave']).sum().reset_index()
    d17['mean'] = d17['mean'] / d17['total_n']

    """
    CONSTANTS
    """
    LABS = d.Lab.unique().tolist()
    WAVE_COLOR = [
        '#D45E00', # '#BA4338', # early
        '#0072B2', # late
        'black'
    ]
    
    """
    PLOT
    """
    titleX=-60    
    opacity=0.7
    
    """
    LABS
    """
    LAB_DROPDOWN = alt.binding_select(options=LABS)
    LAB_SELECTION = alt.selection_single(fields=["Lab"], bind=LAB_DROPDOWN, init={"Lab": LABS[0]}, name="Select")
    
    line_m = alt.Chart(
        d
    ).mark_line(
        size=4, opacity=opacity, point=False
    ).encode(
        x=alt.X('days_since_positive:N', title='Days Since Positive'),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Mean Lab Value', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('Wave:N', scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR))
    ).properties(
        width=200,
        height=200
    )
    
    point_m = alt.Chart(
        d
    ).mark_point(
        opacity=opacity, filled=True
    ).encode(
        x=alt.X('days_since_positive:N', title='Days Since Positive'),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Mean Lab Value', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('Wave:N', scale=alt.Scale(range=WAVE_COLOR)),
        size=alt.Size('total_n:Q', title="# of Patients")
    )
    
    line_m = (line_m + point_m).add_selection(
        LAB_SELECTION
    ).transform_filter(
        LAB_SELECTION
    )
    
    """
    Day 1-7 Average
    """
    bar_m = alt.Chart(
        d17
    ).mark_bar(
        size=32,
        stroke='black'
    ).encode(
        x=alt.X('Wave:N', title='Wave'),
        y=alt.Y('mean:Q', title='Day1-7 Mean Lab Value', axis=alt.Axis(ticks=False, labels=False, domain=False, orient='left')),
        color=alt.Color('Wave:N', scale=alt.Scale(range=WAVE_COLOR))
    ).properties(
        width=100
    ).add_selection(
        LAB_SELECTION
    ).transform_filter(
        LAB_SELECTION
    )
    
    text = alt.Chart(
        d17
    ).mark_text(size=16, dx=-6, dy=0, color='white', baseline='middle', align='right', angle=270, fontWeight=500).encode(
        x=alt.X('Wave:N', title='Wave'),
        y=alt.Y('mean:Q', title='Day1-7 Mean Lab Value', axis=alt.Axis(ticks=False, labels=False, domain=False, orient='right')),
        text=alt.Text('mean:Q')
    ).transform_filter(
        LAB_SELECTION
    )
#     .transform_filter(
#         (f'datum.p_{patient_group} > 0.10')
#     )
#     bar_m = (bar_m + text)
    
#     """
#     # PATIENTS
#     """
#     bar_n = alt.Chart(
#         d
#     ).mark_bar(
#         size=40,
#     ).encode(
#         x=alt.X('days_since_positive:N', title=None),
#         y=alt.Y('total_n:Q', title='# of Patients', axis=alt.Axis(titleX=titleX)),
#         color=alt.Color('Wave:N', scale=alt.Scale(range=WAVE_COLOR))
#     ).properties(
#         width=250,
#         height=250
#     ).add_selection(
#         LAB_SELECTION
#     ).transform_filter(
#         LAB_SELECTION
#     )
        
    plot = (
        alt.hconcat(line_m, bar_m, spacing=20).resolve_scale(y='shared')
    ).properties(
            title={
                "text": f"Mean Lab Values of {'All' if patient_group == 'all' else 'Ever Severe'} Patients by Wave",
                "dx": 80,
                "subtitle": get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False), 
                "subtitleColor": "gray",
            }
        )
    
#     plot = apply_theme(
#         plot,
#         axis_y_title_font_size=16,
#         title_anchor='middle',
#         legend_orient='right'
#     )
#     print(d17)
    return plot

In [ ]:
LABS = df.Lab.unique().tolist()

for i, lab in enumerate(LABS):
    res = alt.hconcat(
        plot(df, patient_group='all', lab=lab), 
        plot(df, patient_group='ever', lab=lab), 
        spacing=50
    ).resolve_scale(y='shared', color='independent', size='independent')

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right'
)
res

In [ ]:
cdf = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_values_standardized_bycountry.csv"))

cdf

In [ ]:
def plot_country(_d, patient_group='all', lab=None):
    d = _d.copy()
    
    """
    DATA PREPROCESSING...
    """
    d.wave = d.wave.apply(lambda x: { 'early': 'Early', 'late': 'Late' }[x])
    d = d[d.scale == 'original']
    d = d[d.setting != 'never']
    d = d[d.setting == patient_group]
    d = d.rename(columns={
        'wave': 'Wave'
    })
    d = d[d.country != 'SINGAPORE']
    d.country = d.country.apply(lambda x: x.capitalize() if x != 'USA' else x)
    d = d[(d.days_since_positive == 0) | (d.days_since_positive == 1) | (d.days_since_positive == 7)]
    d.days_since_positive = d.days_since_positive.apply(lambda x: f"Day {x}")
#     print(d)

    """
    CONSTANTS
    """
    LABS = d.Lab.unique().tolist()
    WAVE_COLOR = [
        '#D45E00', # '#BA4338', # early
        '#0072B2', # late
        'black'
    ]
    COUNTRY_COLORS = ['#0072B2', '#E79F00', '#029F73', '#D45E00', '#CB7AA7']
    
    """
    PLOT
    """
    titleX=-60    
    opacity=0.7
    
    """
    LABS
    """
    LAB_DROPDOWN = alt.binding_select(options=LABS)
    LAB_SELECTION = alt.selection_single(fields=["Lab"], bind=LAB_DROPDOWN, init={"Lab": LABS[0]}, name="Select")
    
    line_m = alt.Chart(
        d
    ).mark_line(
        size=4, opacity=opacity, point=True
    ).encode(
        x=alt.X('days_since_positive:N', title='Days Since Positive'),
        y=alt.Y('mean:Q', scale=alt.Scale(zero=False), title='Standardized Mean Lab', axis=alt.Axis(titleX=titleX)),
        color=alt.Color('country:N', scale=alt.Scale(range=COUNTRY_COLORS)),
        column=alt.Column('Wave:N', header=alt.Header(labelOrient="bottom", title='Wave', titleOrient="bottom", labels=True))
    ).properties(
        width=200,
        height=200
    )
    
    line_m = (line_m).add_selection(
        LAB_SELECTION
    ).transform_filter(
        LAB_SELECTION
    )
        
    plot = (
        alt.hconcat(line_m, spacing=20).resolve_scale(y='shared')
    ).properties(
            title={
                "text": f"Country-Level Lab Values of {'All' if patient_group == 'all' else 'Ever Severe'} Patients by Wave",
                "dx": 80,
                "subtitle": get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False), 
                "subtitleColor": "gray",
            }
        )
    
#     plot = apply_theme(
#         plot,
#         axis_y_title_font_size=16,
#         title_anchor='middle',
#         legend_orient='right'
#     )
#     print(d17)
    return plot

In [ ]:
LABS = df.Lab.unique().tolist()
lab = LABS[0]
res = alt.hconcat(
        plot_country(cdf, patient_group='all', lab=lab), 
        plot_country(cdf, patient_group='ever', lab=lab), 
        spacing=50
    ).resolve_scale(y='shared', color='independent', size='independent')

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right'
)
res

In [ ]:
fr = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_FRANCE.csv"))
ge = pd.read_csv(join("..", "data", "1.1.resurgence", "labs", "lab_pvals_GERMANY.csv"))

pdf = fr.append(ge)
pdf = pdf.drop(columns=['Unnamed: 0'])
pdf = pdf[pdf.scale == 'original']
pdf = pdf[pdf.setting != 'never']

pdf